In [49]:
from elasticsearch import Elasticsearch

from datasets import load_from_disk

import time
import json
from contextlib import contextmanager

### 참고
https://amboulouma.com/elasticsearch-python

https://gh402.tistory.com/51

https://jvvp.tistory.com/1152

In [7]:
# Load wikipedia json data
with open('../../data/wikipedia_documents.json', 'r', encoding='utf-8') as f:
    wiki = json.load(f)

wiki_contents = list(dict.fromkeys([v['text'] for v in wiki.values()]))

In [8]:
# Check wikipedia data
print('Original data length:', len(wiki.keys()))
print(wiki['0'])
print()
print('W/o replicated data length:', len(wiki_contents))
print(wiki_contents[0])

Original data length: 60613
{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '나라 목록', 'author': None, 'html': None, 'document_id': 0}

W/o replicated data length: 56737
이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.
# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.

두 목록은 모두 가나다 순이다.

일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은

In [2]:
es = Elasticsearch('http://localhost:30001')

In [3]:
# Test Elasticsearch Connection
!curl -XGET localhost:30001

{
  "name" : "iCfG3OW",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ne-ovptgRn-Gbd331afwZw",
  "version" : {
    "number" : "5.4.3",
    "build_hash" : "eed30a8",
    "build_date" : "2017-06-22T00:34:03.743Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.1"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
# Check Cluster status
!curl -XGET localhost:30001/_cat/health?v

epoch      timestamp cluster       status node.total node.data shards pri relo init unassign pending_tasks max_task_wait_time active_shards_percent
1651550199 03:56:39  elasticsearch yellow          1         1      5   5    0    0        5             0                  -                 50.0%


In [16]:
# Get Index LIst
!curl -XGET localhost:30001/_cat/indices?v

health status index uuid pri rep docs.count docs.deleted store.size pri.store.size


In [17]:
# Create Index (이미 해당 index 가 존재하면 에러남)
index = wiki
if es.indices.exists(index):
    es.indices.create(index=index)

{'acknowledged': True, 'shards_acknowledged': True}

In [41]:
# Get Index LIst  ->  생성한 index 가 보여야 함!
!curl -XGET localhost:30001/_cat/indices?v

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   wiki  KYxH_TyOSZycjioUq5_7jA   5   1          0            0     66.5kb         66.5kb


In [ ]:
# Index 삭제
# es.indices.delete(index='wiki')

In [42]:
# Insert a data (Index should be existed!)
for idx, text in enumerate(wiki_contents):
    body = {'text': text}
    es.index(index="wiki", doc_type="news", id=idx+1, body=body)


In [47]:
# Get the data
es.get(index='wiki', doc_type='news', id=len(wiki_contents))

{'_index': 'wiki',
 '_type': 'news',
 '_id': '56737',
 '_version': 1,
 'found': True,
 '_source': {'text': '이 협약은 부당노동행위 제도를 규율하고 있다. 협약 제1조에서 반노동조합 차별행위로부터의 보호를 규정하고, 특히 노동조합에 가입하지 않거나 노동조합에서 탈퇴할 것을 조건으로 고용하는 황견계약과 노동조합원이라거나 노동조합 활동을 했다는 이유로 불이익조치를 취하는 것에 대한 보호를 규정하고 있다.\n\n제2조에서 노동자단체와 사용자단체 사이의 상호간 간섭으로부터 충분한 보호를 하도록 규정한다. 사용자 또는 사용자단체의 지배 하에 둘 목적으로 노동자단체의 설립을 지원하거나 노동자단체에 재정적으로 또는 그 밖의 방법으로 지원하는 것은 간섭행위로 보게 된다.\n\n노동조건을 단체협약으로 규율하도록, 사용자 및 사용자단체와 노동자단체 사이의 자발적 교섭을 위한 기구를 발전시키고 이용을 촉진하는 규정을 담았다.'}}

In [36]:
# Delete a data
# es.delete(index='wiki', id=1)

{'took': 61,
 'timed_out': False,
 'total': 3,
 'deleted': 3,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [28]:
# Search the data by Query (Main !!!!)
body = {
    'query': {
        'match': {
            'text': '유엔 국제 기구'
        }
    }
}

res = es.search(index='wiki', body=body)

In [51]:
org_dataset = load_from_disk('../../data/train_dataset')

org_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [54]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

In [74]:
# org_dataset['validation']['question']
for i in range(21, 101):
    with timer(f'TOP K: {i}'):
        TOPK = i
        doc_scores = []
        doc_indices = []
        for j in range(len(org_dataset['validation'])):
            body = {
                'size': TOPK,
                'query': {
                    'match': {
                        'text': org_dataset['validation']['question'][j]
                    }
                }
            }
            res = es.search(index='wiki', body=body)

            a_result_scores = []
            a_result_indices = []

            for item in res['hits']['hits']:
                a_result_scores.append(item['_score'])
                a_result_indices.append(int(item['_id'])-1)

            doc_scores.append(a_result_scores)
            doc_indices.append(a_result_indices)

        correct = 0
        for idx, doc_indice in enumerate(doc_indices):
            for jdx, indice in enumerate(doc_indice):
                if org_dataset['validation']['context'][idx] == wiki_contents[indice]:
                    correct += 1
        
        print(f"Total Validation Score: {correct/len(org_dataset['validation'])*100}%")

Total Validation Score: 65.83333333333333%
[TOP K: 21] done in 10.072 s
Total Validation Score: 66.25%
[TOP K: 22] done in 10.561 s
Total Validation Score: 66.25%
[TOP K: 23] done in 10.993 s
Total Validation Score: 66.66666666666666%
[TOP K: 24] done in 11.413 s
Total Validation Score: 67.08333333333333%
[TOP K: 25] done in 11.900 s
Total Validation Score: 67.08333333333333%
[TOP K: 26] done in 12.447 s
Total Validation Score: 67.5%
[TOP K: 27] done in 12.899 s
Total Validation Score: 67.91666666666667%
[TOP K: 28] done in 13.416 s
Total Validation Score: 67.91666666666667%
[TOP K: 29] done in 13.721 s
Total Validation Score: 67.91666666666667%
[TOP K: 30] done in 14.231 s
Total Validation Score: 67.91666666666667%
[TOP K: 31] done in 14.783 s
Total Validation Score: 68.33333333333333%
[TOP K: 32] done in 15.181 s
Total Validation Score: 68.33333333333333%
[TOP K: 33] done in 15.659 s
Total Validation Score: 68.75%
[TOP K: 34] done in 15.984 s
Total Validation Score: 68.75%
[TOP K: 35